# Motivation
Now that I've got a preliminary method to detect Boggle tiles (from **`03. Finalizing Board Detection`**), I want to try and test it on the data that I've got. 

# Setup
The cells below will set up the rest of this notebook. 

First, I'll configure the kernel: 

In [ ]:
# Change directories to the root of the project
%cd ..

# Enable autoreload of modules
%load_ext autoreload
%autoreload 2

Next, I'll import some relevant libraries:

In [ ]:
# Import statements
import cv2
import pandas as pd
from pathlib import Path
import math
from matplotlib import pyplot as plt
import numpy as np
from statistics import mode
import utils
import cv2
import pytesseract
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Importing custom modules
import utils.board_detection as board_detect

# Set up an EasyOCR reader
import easyocr
reader = easyocr.Reader(['en'])

# Loading Data
Here, I'm going to load in all of the pictures, as well as some information about each of them. 

In [ ]:
# Open the .csv file containing the labeled boards
board_data_df = pd.read_csv("data/labeled-boards.csv")

# Add a column which is the parsed letter sequence
board_data_df["parsed_letter_sequence"] = board_data_df["letter_sequence"].apply(
    lambda letter_list: letter_list.split(";")
)

# Load all of the images using cv2
file_path_to_image = {}
for row in board_data_df.itertuples():
    file_path_to_image[row.file_path] = cv2.imread(row.file_path)

# Parsing Boards
Below, I'm going to run each of the boards through a "parsing" method. 

In [ ]:
# We'll collect some results about the board data here
all_parsed_boards_df_records = []

# Iterate through all of the rows in the board data
for row in tqdm(list(board_data_df.query("difficulty == 'easy'").itertuples())):
    # Try and parse the board
    error_msg = None
    try:
        parsed_board_df = board_detect.parse_boggle_board(
            file_path_to_image[row.file_path],
            max_image_height=1200,
            easyocr_reader=reader
        )

        letter_sequence = list(parsed_board_df["letter"])

    except Exception as e:
        error_msg = str(e)
        letter_sequence = None

    # Add some information to the all_parsed_boards_df_records
    all_parsed_boards_df_records.append(
        {
            "file_path": row.file_path,
            "letter_sequence": letter_sequence,
            "error_msg": error_msg,
        }
    )

# Parse the results into a dataframe
all_parsed_boards_df = pd.DataFrame(all_parsed_boards_df_records)

# Validating Results
Now that I've got the boards parsed, I want to spend some time validating the results. 

In [ ]:
# Merge the DataFrames together
results_to_validate_df = board_data_df.merge(
    all_parsed_boards_df.rename(
        columns={"letter_sequence": "predicted_letter_sequence"}
    ),
    on="file_path",
    how="inner",
)


def match_letter_sequence(sequence_one, sequence_two):
    """
    This function will compare two letter sequences and return a list of
    booleans indicating whether the letters match or not. Each index of the
    list will correspond to a letter in the sequence.
    """

    # If either of the sequences are empty or None, return None
    if not sequence_one or not sequence_two:
        return None

    # If the sequences are not the same length, return None
    if len(sequence_one) != len(sequence_two):
        return None

    # We'll store the results in a list
    results = []

    # Iterate through all of the letters in the sequence
    for index, letter in enumerate(sequence_one):
        other_letter = sequence_two[index]
        results.append(letter == other_letter)

    # Return the results
    return results


results_to_validate_df["letter_sequence_match"] = results_to_validate_df.apply(
    lambda row: match_letter_sequence(
        row["parsed_letter_sequence"], row["predicted_letter_sequence"]
    ),
    axis=1,
)

# Add a column indicating the percent of letters that match
results_to_validate_df["percent_match"] = results_to_validate_df.apply(
    lambda row: sum(row["letter_sequence_match"]) / len(row["letter_sequence_match"])
    if row["letter_sequence_match"]
    else None,
    axis=1,
)

# Add a column indicating which letters don't match
results_to_validate_df["errors"] = results_to_validate_df.apply(
    lambda row: [
        {
            "actual": row.parsed_letter_sequence[letter_idx],
            "predicted": row.predicted_letter_sequence[letter_idx],
            "idx": letter_idx
        }
        for letter_idx, letter_match in enumerate(row.letter_sequence_match)
        if not letter_match
    ]
    if row.letter_sequence_match
    else None,
    axis=1,
)

# Add a column indicating how many errors there are 
results_to_validate_df["num_errors"] = results_to_validate_df.apply(
    lambda row: len(row.errors) if row.errors else None,
    axis=1,
)

In [ ]:
results_to_validate_df["num_errors"].mean()